In [7]:
#Utilização dos algoritmos KNN e LVQ
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [8]:
df = pd.read_csv("https://raw.githubusercontent.com/irvin-s/in-1166-smd/main/missao_2/data/crx.data", sep=',')